In [1]:
import pickle
from pathlib import Path
import pandas as pd 
import os
import sys

In [2]:
import data_set_generate_helper_functions as ghf

In [3]:
sys.executable

'/anaconda3/bin/python3'

In [4]:
sys.path[0]

'/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/5.data_set_counters'

# Path to import the data 

In [8]:
path_to_import = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/0.data_Sets/1.lateral_medial_6"

# Path where to save it 

In [10]:
path_to_save= "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis"

# Update depending on analysis: 

In [11]:
seq_str = '0a1b2a3b4abcd5a6a7a'
HowManyBack=1

# Folder to save the data: 

In [12]:
folderName="right_dual_color_photo_full_data" #folder where the plot data_set will be saved 
#folderName="left_dual_color_photo_full_data_set_after_inspection" #folder where the plot data_set will be saved 

In [13]:
full_path_to_save = path_to_save+"/"+folderName+"/"+seq_str
full_path_to_save

'/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis/right_dual_color_photo_full_data/0a1b2a3b4abcd5a6a7a'

# import the data table: 

In [14]:
#name of xlsx file that has the sessions we want to run: 
MasterSheetName= 'lateral_ medial_6_to_run_analysis'

In [15]:
Mouse_Date_FileName = pd.read_excel(path_to_import+'/'+MasterSheetName+'.xlsx') #mdf is mouse data file 

In [ ]:
Mouse_Date_FileName

In [ ]:
#Mouse_Date_FileName = Mouse_Date_FileName.iloc[0:45,:]

In [ ]:
#Mouse_Date_FileName

# Import the full data structure for each session: 

In [ ]:
def extract_data_object_data_lick(Mouse_Date_FileName,path):
    data_lick_data_set = []
    counter = 1 
    
    l_mouse = list(Mouse_Date_FileName["mouse"]) 
    l_date_ = list(Mouse_Date_FileName.apply(lambda x: generate_date_(x["date"]),axis=1))  
    l_date_and_name = list(Mouse_Date_FileName.apply(lambda x: generate_name_and_date(x["date"],x["mouse"]),axis=1))
    
    numer_of_sessions_dataset = len(l_mouse)
    
    for (mouse, date_, date_and_name) in zip(l_mouse, l_date_, l_date_and_name):
        print ("session number"+" "+str(counter)+" "+"was imported (out of"+" "+str(numer_of_sessions_dataset)+")")
        data_lick = pd.read_excel(path+"/"+mouse+"/"+date_and_name+"/"+mouse+"_dataLick_label.xlsx")
        data_lick_data_set.append(data_lick)  
        counter +=1
        
    return (data_lick_data_set)

In [ ]:
def generate_date_(Date):
    return('20'+str(Date)[:2]+'_'+str(Date)[2:4]+'_'+str(Date)[4:6])

def generate_name_and_date(Date,Mouse):
    return ('20'+str(Date)[:2]+'_'+str(Date)[2:4]+'_'+str(Date)[4:6]+'__'+Mouse)

In [ ]:
%%time
behavior_data_lick_data_set = extract_data_object_data_lick(Mouse_Date_FileName,path_to_import)

extract_data_object_data_lick(Mouse_Date_FileName,path)

In [ ]:
# behavior_data_lick_data_set[0].to_csv("testing2.csv")

# A function to help loop on each session with various functions 

In [ ]:
def session_looper(data_set,function):
    organized_data_lick_list=[]
    for session in data_set:
        result = function(session)
        organized_data_lick_list.append(result)
    return organized_data_lick_list

In [ ]:
def clean_behavior_data_lick (behavior_data_lick):        
    if behavior_data_lick.columns.tolist() == ['iBlock', 'iTrial', 'iSpout', 'sTrial_start', 'sTrial_end', 'tState']:
        pass
    else:
        #Shifting the names on the columns, so each column has the right label. (the raw data has these labels shifted)
        behavior_data_lick=behavior_data_lick.rename(columns={"nTrial": "iBlock", "iBlock":"iTrial", "iTrial":"iSpout","iSpout":"sTrial_start", "sTrial_start":"sTrial_end", "sTrial_end":"tState_new"})
        #I will delete the extra column
        behavior_data_lick=behavior_data_lick.drop("tState", 1)
        behavior_data_lick=behavior_data_lick.rename(columns={"tState_new":"tState"})
    #this is to delete the first and last rows which have no lick data  
    
    behavior_data_lick=behavior_data_lick[1: -1]
    #Drop everything before first trial
    behavior_data_lick= behavior_data_lick[behavior_data_lick.iBlock>0]
    return (behavior_data_lick)

In [ ]:
behavior_data_lick_data_set_clean = session_looper (behavior_data_lick_data_set,clean_behavior_data_lick)

In [ ]:
data_set = behavior_data_lick_data_set_clean.copy()

In [ ]:
# data_set[0].to_csv("testing.csv")

In [ ]:
# %%time
# root = Path(full_path_to_save)
# d = "data_object_data_lick"+"_"+seq_str+'.pickle'
# my_path = root / d 
# print (my_path)
# my_file = open(my_path, 'wb')
# my_file = pickle.dump((data_set),my_file)

In [ ]:
df_with_states = pd.read_csv("/Users/gilmandelbaum/Desktop/stateList.csv", header=None,names = ["state #","description"])

In [ ]:
def unique_list_of_state_transitions(behavior_data_lick_df_data_set):
    
    st_transi_data_set= []
    count = 0

    for behavior_data_lick_df in behavior_data_lick_df_data_set:
        
        behavior_data_lick_df["state_tuple"] = behavior_data_lick_df.apply(lambda x: (x["sTrial_start"], x["sTrial_end"]), axis=1)
        state_transition_unique_session = list(set(behavior_data_lick_df["state_tuple"]))
        st_transi_data_set.extend(state_transition_unique_session)
        
    
    st_transi_uni_data_set = list(set(st_transi_data_set))
    st_transi_uni_data_set_sorted = sorted(st_transi_uni_data_set, key=lambda x: x[0])
    st_transi_uni_data_set_sorted_names = pd.DataFrame(st_transi_uni_data_set_sorted, columns = ["sTrial_start #","sTrial_end #"])
    
    
    return (st_transi_uni_data_set_sorted_names)

In [ ]:
%%time
st_transi_uni_df = unique_list_of_state_transitions(data_set)

In [ ]:
st_transi_uni_df

In [ ]:
st_transi_uni_df['Trial_start_description']=st_transi_uni_df['sTrial_start #']
st_transi_uni_df['Trial_end_description']=st_transi_uni_df['sTrial_end #']

state_num_list=list(df_with_states['state #'])
state_description_list=list(df_with_states['description'])

st_transi_uni_df['Trial_start_description'].replace(to_replace=state_num_list, value=state_description_list, 
                                                    inplace=True)
st_transi_uni_df['Trial_end_description'].replace(to_replace=state_num_list, value=state_description_list, 
                                                    inplace=True)

In [ ]:
st_transi_uni_df

In [ ]:
def count_state_transitions_per_session(behavior_data_lick_df_data_set,st_transi_uni_df,Mouse_Date_FileName):
    
    st_transi_uni_df_list = list(st_transi_uni_df.apply(lambda x: (x["sTrial_start #"], x["sTrial_end #"]), axis=1))
    l_date_and_name = list(Mouse_Date_FileName.apply(lambda x: generate_name_and_date(x["date"],x["mouse"]),axis=1))

    for behavior_data_lick_df, date_and_name in zip(behavior_data_lick_df_data_set,l_date_and_name):
        counts_per_session_list = []
        state_transition_session_list = list(behavior_data_lick_df.apply(lambda x: (x["sTrial_start"], x["sTrial_end"]), axis=1))
        
        for tuple_states in st_transi_uni_df_list:
            count = state_transition_session_list.count(tuple_states)
            counts_per_session_list.append(count)
        
        st_transi_uni_df[date_and_name] = counts_per_session_list
    
    return (st_transi_uni_df)

In [ ]:
def count_states_occurance(st_transi_uni_df,uni_trans_list,data_set,session_list):
    for session,date in zip(data_set,session_list):
        trans_count_list = []
        for trans in uni_trans_list:
            session_state_occurance = len(session[session['state_tuple']==trans])
            trans_count_list.append(session_state_occurance)
        st_transi_uni_df[date]=trans_count_list
            
    return st_transi_uni_df

In [ ]:
%%time

st_transi_uni_df_num_licks_per_session = count_state_transitions_per_session(data_set,
                                                                              st_transi_uni_df,
                                                                              Mouse_Date_FileName)

In [ ]:
st_transi_uni_df_num_licks_per_session.head()

In [ ]:
st_transi_uni_df_num_licks_per_session.iloc[:,4:]

In [ ]:
st_transi_uni_df_num_licks_per_session ["total_number_of_licks_state_transi"] = st_transi_uni_df_num_licks_per_session.iloc[:,4:].sum(axis=1)

In [ ]:
st_transi_uni_df_num_licks_per_session.to_csv("testing.csv")